### CRF

基于sklearn_crfsuite NER系统搭建，本例来自于sklearn_crfsuite官方tutorial

在anaconda的命令行窗口做操作，系统环境中没有这个  
创建沙箱环境  
https://blog.csdn.net/weixin_42218868/article/details/95398043  
在沙箱环境中做好配置，整理.py文件，并做后续处理。

In [1]:
# 导入相关库
import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

# sklearn-crfsuite的文档中的信息来看，更新到2017年，支持到python3.6，用不了
# 尝试用沙箱解决
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

ModuleNotFoundError: No module named 'sklearn_crfsuite'

In [ ]:
# 基于NLTK下载示例数据集
nltk.download('conll2002')

In [ ]:
# 设置训练和测试样本
train_sents = list(nltk.corpus.conll2002.iob_sents('esp.train'))
test_sents = list(nltk.corpus.conll2002.iob_sents('esp.testb'))

In [ ]:
train_sents[0]

In [ ]:
# 单词转化为数值特征
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [ ]:
sent2features(train_sents[0])[0]

In [ ]:
# 构造训练集和测试集
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

In [ ]:
print(len(X_train), len(X_test))

In [ ]:
# 创建CRF模型实例
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
# 模型训练
crf.fit(X_train, y_train)
# 类别标签
labels = list(crf.classes_)
labels.remove('O')
# 模型预测
y_pred = crf.predict(X_test)
# 计算F1得分
metrics.flat_f1_score(y_test, y_pred,
                      average='weighted', labels=labels)

In [ ]:
# 打印B和I组的模型结果
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))